In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)
BATCH_SIZE = 32
IMAGE_DIMS = (256, 256, 3)

2024-12-19 15:25:47.954088: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 15:25:48.265794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


In [2]:
train_dir = r'/home/arsenal/ml/OCT2017/train'
val_dir = r'/home/arsenal/ml/OCT2017/val'

train_dataset = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256,256),
    batch_size=BATCH_SIZE
)

val_dataset = keras.preprocessing.image_dataset_from_directory(
    val_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256,256),
    batch_size=BATCH_SIZE
)

class_names = train_dataset.class_names
print("Class names:", class_names)

Found 83489 files belonging to 4 classes.
Using 66792 files for training.


2024-12-19 15:25:53.118903: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-19 15:25:53.196678: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-19 15:25:53.196710: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-19 15:25:53.198868: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-19 15:25:53.198896: I external/local_xla/xla/stream_executor

Found 32 files belonging to 4 classes.
Using 6 files for validation.
Class names: ['CNV', 'DME', 'DRUSEN', 'NORMAL']


In [3]:
import tensorboard

model1 = keras.models.Sequential([
    keras.layers.Input(shape=IMAGE_DIMS),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

model2 = keras.models.Sequential([
    keras.layers.Input(shape=IMAGE_DIMS),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model2.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

input = keras.layers.Input(shape=IMAGE_DIMS)

model1_output = model1(input)
model2_output = model2(input)

combined_output = keras.layers.Concatenate()([model1_output, model2_output])
final_model = keras.Model(inputs=input, outputs=combined_output)

tensorboard_callback = keras.callbacks.TensorBoard('assets/logs/multiModel')

final_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

final_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1)         │ 31,509,697 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 3)         │ 31,509,955 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 4)         │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ sequential_1[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 63,019,652 (240.40 MB)

 Trainable params: 63,019,652 (240.40 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def one_hot_encode(dataset, num_classes):
    def encode(image, label):
        label = tf.one_hot(label, num_classes)
        return image, label
    return dataset.map(encode)

num_classes = 4

train_dataset_encoded = one_hot_encode(train_dataset, num_classes)
val_dataset_encoded = one_hot_encode(val_dataset, num_classes)

if not final_model.built:
    final_model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

final_model.build(input_shape=(None, *IMAGE_DIMS))

history = final_model.fit(train_dataset_encoded,
                          epochs=10,
                          validation_data=val_dataset_encoded,
                          callbacks=[tensorboard_callback])


Epoch 1/10


I0000 00:00:1734621955.840729     576 service.cc:145] XLA service 0x7f5824006510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734621955.840767     576 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2024-12-19 15:25:55.938407: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-19 15:25:56.234291: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1734621960.954470     576 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2088/2088 ━━━━━━━━━━━━━━━━━━━━ 209s 97ms/step - accuracy: 0.4460 - loss: 7.1591 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 2/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 195s 93ms/step - accuracy: 0.4459 - loss: 7.1672 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 3/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 198s 95ms/step - accuracy: 0.4461 - loss: 7.1686 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 4/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 197s 94ms/step - accuracy: 0.4462 - loss: 7.1646 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 5/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 198s 95ms/step - accuracy: 0.4464 - loss: 7.1664 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 6/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 195s 93ms/step - accuracy: 0.4461 - loss: 7.1662 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 7/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 200s 96ms/step - accuracy: 0.4462 - loss: 7.1646 - val_accuracy: 0.3333 - val_loss: 8.4056
Epoch 8/10
2088/2088 ━━━━━━━━━━━━━━━━━━━━ 196s 94ms/step - accuracy: 0.4461 - lo

In [11]:
%pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
%tensorboard --logdir=assets/logs/multiModel